How has the distribution of participating teams from across the country changed since 2010? 
How have non-california teams performed since 2010?
Do the same teams dominate both girls and boys categories?

In [1]:
from tabula import read_pdf
import pandas as pd

In [2]:
import PyPDF2

In [3]:
import os

In [4]:
import re
import numpy as np

In [5]:
pd.set_option('display.max_rows', 500)

## Import and clean pdf files
Import and clean using a function that works with the format of the majority of the files.

In [135]:
folder_path = "/home/jessie/Documents/Data"

files = os.listdir(folder_path)
files = [x for x in files if ('.pdf' in x ) & ~('2018' in x) & (('Boys' in x) | ('Girls' in x)) & ~('2012 Girls' in x)]

In [136]:
files

['2016 Boys.pdf',
 '2013 Girls.pdf',
 '2012 Boys.pdf',
 '2013 Boys.pdf',
 '2017 Boys.pdf',
 '2015 Girls.pdf',
 '2014 Boys.pdf',
 '2017 Girls.pdf',
 '2014 Girls.pdf',
 '2015 Boys.pdf',
 '2016 Girls.pdf']

In [139]:
def clean_df(df, filename):
    df = df.set_index('Finish')

    df = df.dropna(axis=0, how='all')

    cols = df.columns.to_series()
    cols = cols.dropna()
    filtered_cols = cols[~cols.str.contains(r'unnamed', flags=re.I)]

    df = df.dropna(axis=1, how='all')
    df.columns = filtered_cols

    #move second set of columns to top
    grps = (df.index == 'Finish').cumsum()
    dfs = []
    for grp, dfg in df.groupby(grps):
        if grp != 0:
            dfg.columns = dfg.iloc[0].values
            dfs.append(dfg)
        else: dfs.append(dfg)

    df_new = pd.concat(dfs, axis=1, sort=False)
    df_new = df_new.drop('Finish', axis=0)
    df_new = df_new.dropna(axis=0, how='all')

    cols = df_new.columns.to_series()
    cols = cols.dropna()
    filtered_cols = cols[~cols.str.contains(r'unnamed', flags=re.I)]

    df_new = df_new.loc[:, df_new.columns.notnull()]
    df_new.columns = filtered_cols

    df_new = df_new.apply(lambda col: col.str.upper())

    df_stacked = df_new.stack().reset_index()
    df_stacked = df_stacked.rename(columns = {'level_0':'Finish', 'level_1':'Category', 0:'Team'})
    df_stacked['Age'] = df_stacked['Category'].str.extract(r'([0-9]{2})')
    df_stacked['Sex'] = df_stacked['Category'].str.extract(r'\b(\w+)$')
    df_stacked = df_stacked.drop('Category', axis=1)
    df_stacked['Year'] = filename
    df_stacked['Year'] = df_stacked['Year'].str.extract('^([\w\-]+)')
    
    return df_stacked

In [140]:
data = pd.DataFrame(columns=['Finish', 'Team', 'Age', 'Sex', 'Year'])

for pdf in files:
    
    df = read_pdf(pdf, stream=True, pages='all')
    df_clean = clean_df(df,pdf)
    data = pd.concat([data, df_clean], axis=0, sort=False)

In [141]:
data['Year'].value_counts()

2015    704
2017    699
2016    642
2014    589
2013    558
2012    300
Name: Year, dtype: int64

## Import and clean remaining pdf files
A few of the files have a different format and required slightly different functions to import and clean.

In [143]:
other_files = ['2010.pdf', '2011.pdf', '2012 Girls.pdf']

In [144]:
def clean_df_other(df, filename):
    df = df.set_index('Finish')
    grps = (df.index == 'Finish').cumsum()

    dfs = []
    for grp, dfg in df.groupby(grps):
        if grp != 0:
            dfg.columns = dfg.iloc[0].values
            dfs.append(dfg)
        else: dfs.append(dfg)
        
    df_new = pd.concat(dfs, axis=1, sort=False)
    df_new = df_new.drop('Finish', axis=0)
    df_new = df_new.dropna(axis=0, how='all')

    cols = df_new.columns.to_series()
    cols = cols.dropna()
    filtered_cols = cols[~cols.str.contains(r'unnamed', flags=re.I)]
    #filtered_cols = filtered_cols[~filtered_cols.str.contains(r'21', flags=re.I)]

    df_new = df_new.dropna(axis=1, how='all')
    df_new.columns = filtered_cols

    df_new = df_new.apply(lambda col: col.str.upper())
    
    df_stacked = df_new.stack().reset_index()
    df_stacked = df_stacked.rename(columns = {'level_0':'Finish', 'level_1':'Category', 0:'Team'})
    df_stacked['Age'] = df_stacked['Category'].str.extract(r'([0-9]{2})')
    df_stacked['Sex'] = df_stacked['Category'].str.extract(r'\b(\w+)$')
    df_stacked = df_stacked.drop('Category', axis=1)
    df_stacked['Year'] = filename
    df_stacked['Year'] = df_stacked['Year'].str.extract('^([\w\-]+)')
    return df_stacked

In [145]:
for pdf in other_files:
    
    df = read_pdf(pdf, stream=True, pages='all')
    df_clean = clean_df_other(df,pdf)
    data = pd.concat([data, df_clean], axis=0, sort=False)

In [146]:
data['Year'].value_counts()

2015    704
2017    699
2016    642
2014    589
2012    559
2013    558
2011    512
2010    352
Name: Year, dtype: int64

In [169]:
files_2018 = os.listdir(folder_path)
files_2018 = [x for x in files_2018 if ('2018' in x)]

In [170]:
files_2018

['2018JOresults-14UG.pdf',
 '2018JOresults-18uB.pdf',
 '2018JOresults-10UG.pdf',
 '2018JOresults-12UG.pdf',
 '2018JOresults-18UG.pdf',
 '2018JOresults-12uB.pdf',
 '2018JOresults-16UG.pdf',
 '2018JOresults-16uB.pdf',
 '2018JOresults-10UM.pdf',
 '2018JOresults-14uB.pdf']

In [176]:
df = read_pdf('2018JOresults-18uB.pdf', pages='all', lattice=True)

In [177]:
df

,18 & Under Boys,Unnamed: 1,Unnamed: 2
0,Flight,Team\rFinish,Team Name
1,NaN,1,UNITED COZY BOYS
2,2,STANFORD A,NaN
3,3,VANGUARD WHITE SENIOR,NaN
4,4,LA PREMIER,NaN
5,5,OCWPC,NaN
6,6,LAMORINDA A,NaN
7,7,SHAQ A,NaN
8,8,CC UNITED A,NaN
9,9,TROJAN CARDINAL A,NaN


## Import and clean excel files
The final files are excel and require a simpler function.

In [178]:
folder_path = "/home/jessie/Documents/Data"

files_2019 = os.listdir(folder_path)
files_2019 = [x for x in files_2019 if ('.xlsx' in x)]

In [ ]:
def df_from_csv(path):
    df = read_csv(path, nrows=1) # read just first line for columns
    columns = df.columns.tolist() # get the columns
    cols_to_use = columns[:len(columns)-1] # drop the last one
    df = read_csv(path, usecols=cols_to_use)
    return df

In [179]:
files_2019

['NJO 2019 Session 2 Final Placing.xlsx',
 'NJO 2019 Session 1 Final Placing.xlsx']

In [205]:
def clean_excel(df, filename):
    df_new = df.drop('Flight', axis=1)
    df_new = df_new.set_index('Finish')
    df_new = df_new.apply(lambda col: col.str.upper())

    df_stacked = df_new.stack().reset_index()
    df_stacked = df_stacked.rename(columns = {'level_1':'Category', 0:'Team'})
    df_stacked['Age'] = df_stacked['Category'].str.extract(r'([0-9]{2})')
    df_stacked['Sex'] = df_stacked['Category'].str.extract(r'\b(\w+)$')
    df_stacked = df_stacked.drop('Category', axis=1)
    df_stacked['Year'] = filename
    df_stacked['Year'] = df_stacked['Year'].str.extract('([0-9]{4})')

    return df_stacked

In [206]:
clean_excel(this, 'NJO 2019 Session 2 Final Placing.xlsx')

,Finish,Team,Age,Sex,Year
0,1,LAMORINDA - BLUE TEAM,10,Girls,2019
1,1,SHORE AQUATICS,10,Mixed,2019
2,1,LAGUNA BEACH WATER POLO A,12,Girls,2019
3,1,SAN DIEGO SHORES,12,Mixed,2019
4,1,SOCAL BLACK (A),14,Girls,2019
5,1,SET,16,Girls,2019
6,1,680 RED (A),18,Girls,2019
7,2,680 RED (A),10,Girls,2019
8,2,VANGUARD AQUATICS,10,Mixed,2019
9,2,LOS ANGELES PREMIER,12,Girls,2019


In [208]:
for file in files_2019:
    df = pd.read_excel(file)
    df_clean = clean_excel(df, file)
    data = pd.concat([data, df_clean], axis=0, sort=False)

In [209]:
data['Year'].value_counts()

2019    875
2015    704
2017    699
2016    642
2014    589
2012    559
2013    558
2011    512
2010    352
Name: Year, dtype: int64